In [1]:
!pip install python-docx PyPDF2

In [2]:
import os
import pandas as pd
import docx
import PyPDF2

In [6]:
#Program 1 – Study of Python and basic commands to access text data
dir_path='C:\\Users\\Pooja\\Downloads\\7th sem\\NLP LAB'
files=[f for f in os.listdir(dir_path) if (f.endswith('.docx') or f.endswith('.txt') or f.endswith('.pdf'))]

data=[]
for txtfile in files:
    if txtfile.endswith('.txt'):
        with open(os.path.join(dir_path,txtfile),'r') as file:
            content=file.read()
            data.append({'filename':txtfile,'content':content})
    
    if txtfile.endswith('.docx'):
            doc_path=os.path.join(dir_path,txtfile)
            doc=docx.Document(doc_path)
            content='\n'.join([paragraph.text for paragraph in doc.paragraphs])
            data.append({'filename':txtfile,'content':content})
    
    if txtfile.endswith('.pdf'):
        with open(os.path.join(dir_path,txtfile),'rb') as f:
            pdf_reader=PyPDF2.PdfReader(f)
            num_pages=len(pdf_reader.pages)
            for page in range(num_pages):
                content=pdf_reader.pages[page].extract_text()
                data.append({'filename':txtfile,'content':content})
res=pd.DataFrame(data)
res

,filename,content
0,documet.docx,Doc version for the first experiment
1,dot.pdf,This is the pdf version for the first experime...
2,nlp.txt,"Text Pre-porcessing,lemmatization,Stemming\nTo..."
3,rand.txt,7th semester Natural Language Processing\nArti...


In [8]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Pooja\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Pooja\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
df=res

In [13]:
#Program 2 – Perform Pre-Proessing
import os
import re
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

ps=PorterStemmer()
stopwords=set(stopwords.words('english'))

def clean_text(text):
    text=re.sub(r'[^A-Za-z\s]','',text)
    text=text.lower()
    tokens=word_tokenize(text)
    tokens=[ps.stem(text) for text in tokens if text not in stopwords]
    
    return ' '.join(tokens) #j2

df['clean']=df['content'].apply(clean_text)
df

,filename,content,clean
0,documet.docx,Doc version for the first experiment,doc version first experi
1,dot.pdf,This is the pdf version for the first experime...,pdf version first experi
2,nlp.txt,"Text Pre-porcessing,lemmatization,Stemming\nTo...",text preporcessinglemmatizationstem tokenizati...
3,rand.txt,7th semester Natural Language Processing\nArti...,th semest natur languag process artifici intel...


In [20]:
data=[['rand.txt','7th semester lab'],['file.txt','natural language processing'],['one.txt','i like reading books']]
df2=pd.DataFrame(data,columns=['filename','content'])
df2

,filename,content
0,rand.txt,7th semester lab
1,file.txt,natural language processing
2,one.txt,i like reading books


In [18]:
#Program 3 – Implement N-Gram Model
from nltk.util import ngrams
  #LIST1
def gen_ngram(text,n):
    text=text.split()
    return [' '.join(gram) for gram in ngrams(text,n)] #j3

df['bigram']=df['clean'].apply(gen_ngram,n=2)
df

,filename,content,clean,bigram
0,documet.docx,Doc version for the first experiment,doc version first experi,"[doc version, version first, first experi]"
1,dot.pdf,This is the pdf version for the first experime...,pdf version first experi,"[pdf version, version first, first experi]"
2,nlp.txt,"Text Pre-porcessing,lemmatization,Stemming\nTo...",text preporcessinglemmatizationstem tokenizati...,"[text preporcessinglemmatizationstem, preporce..."
3,rand.txt,7th semester Natural Language Processing\nArti...,th semest natur languag process artifici intel...,"[th semest, semest natur, natur languag, langu..."


In [27]:
df2['bigram']=df2['content'].apply(gen_ngram,n=1)
df2


,filename,content,bigram
0,rand.txt,7th semester lab,"[7th, semester, lab]"
1,file.txt,natural language processing,"[natural, language, processing]"
2,one.txt,i like reading books,"[i, like, reading, books]"


In [29]:
#Program 4 – Implement POS Tagging
nltk.download('averaged_perceptron_tagger')
import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize

def genpos_tag(text):
    token=word_tokenize(text)
    return pos_tag(token)

df2['pos']=df2['content'].apply(genpos_tag)
df2
    
    

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Pooja\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,filename,content,bigram,pos
0,rand.txt,7th semester lab,"[7th, semester, lab]","[(7th, CD), (semester, NN), (lab, NN)]"
1,file.txt,natural language processing,"[natural, language, processing]","[(natural, JJ), (language, NN), (processing, NN)]"
2,one.txt,i like reading books,"[i, like, reading, books]","[(i, NNS), (like, VBP), (reading, VBG), (books..."


In [42]:
#Program 5 – Implement Chunking
nltk.download('maxent_ne_chuker')   
nltk.download('words')
import nltk

def genchunk(text):
    grammar="NP: {<DT>?<JJ>*<NN>}"
    cp=nltk.RegexpParser(grammar)
    tree=cp.parse(text)  #parse
    
    NP=[]
    for subtree in tree.subtrees():
        if subtree.label() == "NP":
            NP.append(' '.join(word for word, tag in subtree.leaves())) #j5

    return NP

df2['chunk']=df2['pos'].apply(genchunk) #apply on pos
df2

[nltk_data] Error loading maxent_ne_chuker: Package 'maxent_ne_chuker'
[nltk_data]     not found in index
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Pooja\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


,filename,content,bigram,pos,chunk
0,rand.txt,7th semester lab,"[7th, semester, lab]","[(7th, CD), (semester, NN), (lab, NN)]","[semester, lab]"
1,file.txt,natural language processing,"[natural, language, processing]","[(natural, JJ), (language, NN), (processing, NN)]","[natural language, processing]"
2,one.txt,i like reading books,"[i, like, reading, books]","[(i, NNS), (like, VBP), (reading, VBG), (books...",[]


In [37]:
dff=df2.copy()

In [48]:
#Program 6 – Implement Sentence Completion
import random   #LIST2
sentence_prompts={'she opened the door and saw a':['bright light','mysterious figure','beautiful garden'],
                 'afer a long day at work,i like to relax by':['reading a book','watching tv','going for walk']}
input_pr='afer a long day at work,i like to relax by'

if input_pr in sentence_prompts:
    possib=sentence_prompts[input_pr]
    
    print('possible completiosn')
    for comp in possib:
        print(f'- {input_pr} {comp}')
else:
    print('not in dictionary')

possible completiosn
- afer a long day at work,i like to relax by reading a book
- afer a long day at work,i like to relax by watching tv
- afer a long day at work,i like to relax by going for walk


In [49]:
!pip install textblob

  Using cached textblob-0.17.1-py2.py3-none-any.whl (636 kB)


In [50]:
#Program 7 - Sentiment Analysis
from textblob import TextBlob
data=['i love this product!','this is terrible','neutral sentiment']
sentiment=[TextBlob(t).sentiment.polarity for t in data]
label=['positive' if score>0 else 'neagative' if score<0 else 'neutral' for score in sentiment]
d=pd.DataFrame({'data':data,'sentim':sentiment,'label':label})
d

,data,sentim,label
0,i love this product!,0.625,positive
1,this is terrible,-1.000,neagative
2,neutral sentiment,0.000,neutral


In [52]:
!pip install bert-extractive-summarizer
!pip install summarizer
!pip install transformers

In [56]:
#Program 8 - Text Summarisation (Abstractive and Extractive)
 
a1="Lessons of Chemistry is the latest book nominated for the Sunday times bestseller,the protaganist Elizabeth\
is a lab tehcnician at Royal Institue of Chemistry in the late 1960's when gender bias still prevailed.The story starts by\
telling how Elizabeth had achieved her masters degree despite the hurdles in life and goes on to face many more obstacles,that is \
when she encounters Dr.Alex,a well renowned scientist with several publications and accolades to his name,these two strangers\
end up on a journey of self-discovery and laughter.It's later towards the end we see how Elizabeth fights for women's rights\
and is forced to leaver her position as a chemist due to unforseen circumstances and ends up joining a television channel as \
a cook host who motivates several women all around the world to pursue their dreams by teaching them chemistry one molecule\
at a time"

from transformers import T5ForConditionalGeneration,T5Tokenizer
def abstr(text,max_length=150):
#abstractive summary generation
    model_name = "t5-base"
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)

    input_ids=tokenizer.encode("summarize: "+text,max_length=1024,return_tensors='pt',truncation=True)
    print(input_ids)
    summary_ids=model.generate(input_ids,max_length=max_length,num_beams=4,early_stopping=True,length_penalty=2.0)
    print(summary_ids)
    summary=tokenizer.decode(summary_ids[0],skip_special_tokens=True)
    summary2=tokenizer.decode(summary_ids[0])
    print(summary2)
    
    return summary

sumart=abstr(a1)
print(sumart)

C:\Users\Pooja\anaconda3\envs\tf\lib\site-packages\transformers\models\t5\tokenization_t5.py:217: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


tensor([[21603,    10,   622,  6577,    13, 21299,    19,     8,  1251,   484,
           150,  1109,   920,    21,     8,  1771,   648, 31666,     6,   532,
           813,  2408,   152,   343,  9066,   159,     3,     9,  7690,     3,
            17,    15,   107,    75,  7742,   152,    44,  3671,    86,  2248,
            17,    76,    15,    13, 21299,    16,     8,  1480,  8754,    31,
             7,   116,  7285, 14387,   341, 23990,    15,    26,     5,   634,
           733,  3511,    57,  6714,    53,   149,  9066,   141,  5153,   160,
          2325,     7,  1952,     3,  3565,     8, 23463,     7,    16,   280,
            11,  1550,    30,    12,   522,   186,    72, 14637,     6,  6279,
            19,   116,   255,  6326,     7,   707,     5, 27280,     6,     9,
           168,     3,  9623, 17901,    28,   633, 10142,    11, 30022,     7,
            12,   112,   564,     6, 19712,   192, 13037,     7,   989,    95,
            30,     3,     9,  2027,    13,  1044,  

In [57]:
#extractive summary
from summarizer import Summarizer
summarizer=Summarizer()
s=summarizer(a1)
print(s)


Lessons of Chemistry is the latest book nominated for the Sunday times bestseller,the protaganist Elizabethis a lab tehcnician at Royal Institue of Chemistry in the late 1960's when gender bias still prevailed. The story starts bytelling how Elizabeth had achieved her masters degree despite the hurdles in life and goes on to face many more obstacles,that is when she encounters Dr.Alex,a well renowned scientist with several publications and accolades to his name,these two strangersend up on a journey of self-discovery and laughter.


C:\Users\Pooja\anaconda3\envs\tf\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\Pooja\anaconda3\envs\tf\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [71]:
#Program 9 -Perform Named Entity Recognition (NER)
import nltk
nltk.download('punkt')
nltk.download('maxent_ne_chunker')
nltk.download('averaged_perceptron_tagger')
nltk.download('words')


from nltk.chunk import ne_chunk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
text='Barack Obama was born in Hawaii and served as the 44th president of the USA'

token=word_tokenize(text)
pos=pos_tag(token)
ner=ne_chunk(pos)
print(ner)


(S
  (PERSON Barack/NNP)
  (PERSON Obama/NNP)
  was/VBD
  born/VBN
  in/IN
  (GPE Hawaii/NNP)
  and/CC
  served/VBD
  as/IN
  the/DT
  44th/CD
  president/NN
  of/IN
  the/DT
  (ORGANIZATION USA/NNP))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Pooja\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Pooja\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Pooja\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Pooja\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [4]:
#Program 10 - Perform Morphological Analysis
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
nltk.download('punkt')
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer,PorterStemmer

text='the quick brown foxes jumped over the lazy dogs'
tokens=word_tokenize(text)
ps=PorterStemmer()
stem=[ps.stem(w) for w in tokens]
    
lemmatizer=WordNetLemmatizer()
lem=[lemmatizer.lemmatize(w,pos=wordnet.VERB) for w in tokens]
    
print('original ',text)
print('prt',stem)
print('lem',lem)
    

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Pooja\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Pooja\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


original  the quick brown foxes jumped over the lazy dogs
prt ['the', 'quick', 'brown', 'fox', 'jump', 'over', 'the', 'lazi', 'dog']
lem ['the', 'quick', 'brown', 'fox', 'jump', 'over', 'the', 'lazy', 'dog']


In [66]:
#morpheme
word='misunderstanding'
prefixes=['mis']
root='understand'
suffixes=['ing']
d=[]
for prefix in prefixes:
    if word.startswith(prefix):
        d.append(prefix)
        word=word[len(prefix):]
for suffix in suffixes:
    if word.endswith(suffix):
        d.append(suffix)
        word=word[:-len(suffix)]
        
d.append(word)
print(word)
print(d)
        

understand
['mis', 'ing', 'understand']
